In [3]:
import yfinance as yf
import datetime

# Define the NIFTY 50 symbol and time range
nifty_symbol = "^NSEI"  # Yahoo Finance symbol for NIFTY 50
end_date = datetime.datetime.today().date()
start_date = end_date - datetime.timedelta(days=365)

# Download the data
nifty_data = yf.download(nifty_symbol, start=start_date, end=end_date)

# Reset the index to keep the 'Date' as a regular column
nifty_data.reset_index(inplace=True)

# Display the first few rows of the data
print(nifty_data.head())

# Save the data to a CSV file for further analysis
nifty_data.to_csv("nifty_daily_data_no_index.csv", index=False)
print("NIFTY 50 data saved to 'nifty_daily_data_no_index.csv'.")


[*********************100%***********************]  1 of 1 completed

Price        Date     Adj Close         Close          High           Low  \
Ticker                    ^NSEI         ^NSEI         ^NSEI         ^NSEI   
0      2023-12-04  20686.800781  20686.800781  20702.650391  20507.750000   
1      2023-12-05  20855.099609  20855.099609  20864.050781  20711.150391   
2      2023-12-06  20937.699219  20937.699219  20961.949219  20852.150391   
3      2023-12-07  20901.150391  20901.150391  20941.250000  20850.800781   
4      2023-12-08  20969.400391  20969.400391  21006.099609  20862.699219   

Price           Open  Volume  
Ticker         ^NSEI   ^NSEI  
0       20601.949219  381100  
1       20808.900391  421000  
2       20950.750000  359200  
3       20932.400391  275000  
4       20934.099609  292400  
NIFTY 50 data saved to 'nifty_daily_data_no_index.csv'.


In [5]:
import pandas as pd

# Load the data from the CSV file
nifty_data = pd.read_csv("nifty_daily_data_no_index.csv")

# Ensure the 'Close' column exists
if 'Close' not in nifty_data.columns:
    print("Error: The file does not contain a 'Close' column.")
    exit()

# Convert the 'Close' column to numeric, coercing errors (e.g., non-numeric values)
nifty_data['Close'] = pd.to_numeric(nifty_data['Close'], errors='coerce')

# Drop rows where 'Close' is NaN after conversion
nifty_data.dropna(subset=['Close'], inplace=True)

# Calculate the 14-day RSI
def calculate_rsi(data, window=14):
    delta = data.diff()  # Price difference
    gain = delta.where(delta > 0, 0).rolling(window=window).mean()  # Average gain
    loss = -delta.where(delta < 0, 0).rolling(window=window).mean()  # Average loss
    rs = gain / loss  # Relative strength
    rsi = 100 - (100 / (1 + rs))  # RSI formula
    return rsi

nifty_data['RSI'] = calculate_rsi(nifty_data['Close'])

# Generate signals based on RSI values
nifty_data['Signal'] = nifty_data['RSI'].apply(lambda x: 1 if x > 70 else (-1 if x < 30 else 0))

# Save the updated data to a new CSV file
output_file = "nifty_with_rsi_signal_no_index.csv"
nifty_data.to_csv(output_file, index=False)
print(f"RSI and signal column added. Data saved to '{output_file}'.")

# Display the last few rows to verify the output
print(nifty_data.tail())


RSI and signal column added. Data saved to 'nifty_with_rsi_signal_no_index.csv'.
           Date        Adj Close         Close            High  \
240  2024-11-25  24221.900390625  24221.900391  24351.55078125   
241  2024-11-26          24194.5  24194.500000  24343.30078125   
242  2024-11-27  24274.900390625  24274.900391  24354.55078125   
243  2024-11-28  23914.150390625  23914.150391        24345.75   
244  2024-11-29  24131.099609375  24131.099609  24188.44921875   

                 Low             Open  Volume        RSI  Signal  
240   24135.44921875   24253.55078125  687200  48.594576       0  
241  24125.400390625   24343.30078125  230700  53.755216       0  
242  24145.650390625   24204.80078125  295000  51.225083       0  
243  23873.349609375  24274.150390625  366700  39.057631       0  
244  23927.150390625  23927.150390625  282100  48.654561       0  


In [7]:
import pandas as pd

# Load the updated data from the CSV file
data = pd.read_csv("nifty_with_rsi_signal_no_index.csv")

# Ensure the 'Signal' column exists
if 'Signal' not in data.columns:
    print("Error: The file does not contain a 'Signal' column.")
    exit()

# Calculate the position as the difference between the current and previous signal
data['Position'] = data['Signal'].diff()

# Initialize variables for profit calculation
profit = 0
holding = None  # To track if a position is currently held (price of the last "buy")

# Calculate profit based on Signal column
profits = []  # To store profit for each transaction
for i, row in data.iterrows():
    if row['Signal'] == 1 and holding is None:  # Buy signal
        holding = row['Close']
        profits.append(0)  # No profit recorded on buy
    elif row['Signal'] == -1 and holding is not None:  # Sell signal
        profit += row['Close'] - holding  # Calculate profit
        profits.append(row['Close'] - holding)  # Record profit for this transaction
        holding = None  # Reset holding after sell
    else:
        profits.append(0)  # No transaction, no profit

# Add the profit column
data['Profit'] = profits

# Save the updated data to a new CSV file
output_file = "nifty_with_position_and_profit.csv"
data.to_csv(output_file, index=False)
print(f"Position and profit columns added. Data saved to '{output_file}'.")

# Display the last few rows to verify the output
print(data.tail())


Position and profit columns added. Data saved to 'nifty_with_position_and_profit.csv'.
           Date     Adj Close         Close          High           Low  \
239  2024-11-25  24221.900391  24221.900391  24351.550781  24135.449219   
240  2024-11-26  24194.500000  24194.500000  24343.300781  24125.400391   
241  2024-11-27  24274.900391  24274.900391  24354.550781  24145.650391   
242  2024-11-28  23914.150391  23914.150391  24345.750000  23873.349609   
243  2024-11-29  24131.099609  24131.099609  24188.449219  23927.150391   

             Open  Volume        RSI  Signal  Position  Profit  
239  24253.550781  687200  48.594576       0       0.0     0.0  
240  24343.300781  230700  53.755216       0       0.0     0.0  
241  24204.800781  295000  51.225083       0       0.0     0.0  
242  24274.150391  366700  39.057631       0       0.0     0.0  
243  23927.150391  282100  48.654561       0       0.0     0.0  
